# Transformer
## Прогнозирование следующей буквы текста


Пример прогнозирования очередной буквы текста при помощи механизма внимания.


In [1]:
import matplotlib.pyplot as plt
from   time import perf_counter as tm                  # таймер sec

import torch
import torch.nn as nn

# Загружаем текст

Возьмём фиксированным  алфавит `CHARS` (русский язык) в нижнем регистре, включив в него конец строки `\n` (для стихов):

In [2]:
CHARS  = " .?,абвгдежзийклмнопрстуфхцчшщъыьэюя\n"        # фиксированный алфавит 
charID = { c:i for i,c in enumerate(CHARS) }             # буква в номер

def preprocess(txt):
    """ Буквы не из алфавита заменяем пробелами """
    txt = txt.lower().replace('ё','e').replace('!','.').replace(';',',')
    txt = ''.join( [c if c in CHARS else ' ' for c in txt] )    
    txt = re.sub(' +', ' ', txt)
    txt = txt.replace(' ,', ',').replace(' .', '.').replace(' ?', '?')
    return  re.sub('\n\s+', '\n', txt)
    
def load_Zip(fname = "data/books.zip"):
    """ Загрузить в строку содержмое всех файлов из zip-архива, проведя их препроцессинг """
    txt = ""
    with zipfile.ZipFile(fname) as myzip:
        for fname in myzip.namelist():
            print(fname, end=": ")
            with myzip.open(fname) as myfile:
                st = preprocess ( myfile.read().decode("utf-8") )     
                chars, words = len(st), len(st.split())
                print(chars, " chars, ",  words, "words", f"{chars/words:.2f} ch/w")  
                txt += " " + st
    return txt
    
text = load_Zip("data/saltan.zip")        
#text = load_Zip("data/books.zip")        
        
print(f"chars: {len(CHARS)}, text length: {len(text)} chars")            
print(f"beg:|{text[:100]}|")        
print(f"end:|{text[-100:]}|")    

saltan.txt: 24201  chars,  3995 words 6.06 ch/w
chars: 37, text length: 24202 chars
beg:|  три девицы под окном 
пряли поздно вечерком. 
кабы я была царица, 
говорит одна девица, 
то на вес|
end:|. 
день прошел царя салтана 
уложили спать вполпьяна. 
я там был, мед, пиво пил 
и усы лишь обмочил.|


In [ ]:
class CFG:
    LEN        = 16    # длина последовательности в символах
    STEP       = 16    # букв между началами последовательностей (если LEN == STEP - не перекрываются)    
    NUM        =  8    # число последних букв по которым вычисляется ошибка (NUM <= STEP)
    
    E_DIM      = 16    # размерность эмбединга
    H_DIM      = 128   # размерность скрытого слоя
    NUM_LAYERS = 3     # число слоёв rnn
    

    DROP       = 0     # dropout вероятность перед классификационным слоем
    BATCH      = 1024  # размер батча
    L2         = 1e-5  # L2-регуляризация
    LR         = 1e-2  # скорость обучения
    
    def get(end=", "):
        return "".join([f"{k}:{v}{end}" for k,v in CFG.__dict__.items() if not k.startswith("__") and k != "get"])

In [ ]:
class Transformer(nn.Module):
    """ B - sample in batch, S - sequence position, E - embedding """
    def __init__(self):        
        super(Transformer, self).__init__()
         
        self.emb = nn.Embedding(num_embeddings=len(VOCAB),  embedding_dim=CFG.EMB_DIM,   padding_idx=0)                
        self.pos = nn.Embedding(num_embeddings=CFG.SEQ_LEN, embedding_dim=CFG.EMB_DIM)                
        
        self.encoder = nn.TransformerEncoder( 
            nn.TransformerEncoderLayer(d_model=CFG.EMB_DIM, dim_feedforward = CFG.FF*CFG.EMB_DIM, 
                                       nhead=CFG.HEADS, dropout=CFG.DROPOUT1, batch_first=True), 
            num_layers=CFG.LAYERS)       
        
        self.drop = nn.Dropout(CFG.DROPOUT2)
        self.fc = nn.Linear(CFG.SEQ_LEN*CFG.EMB_DIM, 1)         
        
    def forward(self, x):                   # (B,S)
        x = self.emb(x)                     # (B,S,E)         
        p = self.pos.weight                 # (S,E)   - position encoder        
        
        x = F.normalize(x, dim = -1) + F.normalize(p, dim = -1)
        x = F.normalize(x, dim = -1)
        #x = x + p
        
        x = self.encoder(x)                 # (B,S,E)
        
        x = F.normalize(x, dim = -1)
        #x = torch.bmm(x, x.transpose(1,2))  # (B,S,E) @ (B,E,S) = (B,S,S)   
        
        #x = torch.relu(x)
        x = nn.Flatten()(x)                 # (B,S*E)        
        
        x = self.drop(x)
        x = self.fc(x)                      # (B,)             
        return x                            
    
model = Transformer()    

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

X = torch.zeros((8, CFG.SEQ_LEN), dtype=torch.long, device=device)
X[0,0] = 1
Y = model(X)
print( Y.shape  )
print( Y[0]  )
summary(model)